In [2]:
from Binaries import *
from scipy.optimize import curve_fit

# For VEM Peak

In [3]:
peak_evolution = []
trigger_examples = []
cut_threshold = 30
leading_bins = 20
trailing_bins = 49
multiplicity = 1
dead_time = trailing_bins
combine_files = 30
station_name = "nuria"

mode = "peak"

if mode == "peak":

    x_range = (cut_threshold, 1200)
    i, f = 20, 100
    func = max
    min_vem = 120
    calibration_guess = GLOBAL.q_peak
    std = 10

elif mode == "charge":

    x_range = (cut_threshold, 12000)
    i, f = 20, 70
    func = sum
    min_vem = 1000
    calibration_guess = GLOBAL.q_charge
    std = 200

first_trigger = lambda x : np.argmax(x > cut_threshold)

def gauss(x, mu, A, sigma):

    return A * np.exp( -(x - mu)**2 / (2 * sigma**2) )

def background(x, A, c):

    return A - (x - min_vem) * c

colors = ["steelblue", "orange", "green"]

h1, h2, h3 = [], [], []

for index in range(0, len(os.listdir(f"/cr/tempdata01/filip/iRODS/{station_name}/")), 3):

    for increment in range(combine_files):

        Buffer = RandomTrace(station_name, index + increment)
        
        vem_peak = []

        # print(f"{index + increment}/{Buffer.all_n_files} {Buffer.random_file}: {len(Buffer._these_traces)} traces", end = "\r")

        station = np.array(Buffer._these_traces)
        pmt1, pmt2, pmt3 = station[:,0], station[:,1], station[:,2]
        cut_mask = np.logical_and(np.any(pmt1 > cut_threshold, axis = 1), np.any(pmt2 > cut_threshold, axis = 1), np.any(pmt3 > cut_threshold, axis = 1))

        for (p1, p2, p3) in zip(pmt1[cut_mask], pmt2[cut_mask], pmt3[cut_mask]):

            steps = iter(range(min([first_trigger(p1), first_trigger(p2), first_trigger(p3)]), 2048))
            # STRONG COINCIDENCE
            for step in steps:

                trigger_list = [p1[step] > cut_threshold, p2[step] > cut_threshold, p3[step] > cut_threshold]

                if trigger_list.count(True) >= multiplicity:

                    start, stop = max(0, step - leading_bins), min(2048, step + trailing_bins)

                    p1_data, p2_data, p3_data = p1[start : stop], p2[start : stop], p3[start : stop]
                    h1.append(func(p1_data)), h2.append(func(p2_data)), h3.append(func(p3_data))

                    if len(trigger_examples) < 100: trigger_examples.append([p1_data, p2_data, p3_data])

                    for _ in range(dead_time): next(steps, None)
                    # step += dead_time       # keep iterating
                    # break                     # stop after first hit
        
            # # WEAK COINCIDENCE
            # trigger_bin = np.argmax(p1 > cut_threshold)
            # start = max(0, trigger_bin - leading_bins)
            # stop = min(trigger_bin + trailing_bins, 2048)

            # p1_data, p2_data, p3_data = p1[start : stop], p2[start : stop], p3[start : stop]

            # if np.any(p2_data > cut_threshold) and np.any(p3_data > cut_threshold):

            #     h1.append(func(p1_data)), h2.append(func(p2_data)), h3.append(func(p3_data))

    fig, axes = plt.subplots(3, 1, sharex = True)
    name = Buffer.random_file.split("_")[0]

    try:
        for j, histogram in enumerate([h1, h2, h3]):

            n, bins, _ = axes[j].hist(histogram,  bins = 250, range = x_range, label = f"{len(histogram)} coincidences", ls = "solid", histtype = "step", color = colors[j], lw = 1)
            bin_centers = 0.5 * (bins[1:] + bins[:-1])

            # Signal region
            x, y = bin_centers[i : f], n[i : f]
            x_smooth = np.linspace(min(x), max(x), 100)

            x_out = list(bin_centers[0 : i]) + list(bin_centers[f : -1])
            y_out = list(n[0 : i]) + list(n[f : -1])
            x_out_smooth = np.linspace(min(x_out), max(x_out), 100)

            # fitting background exponential first
            y_out_log = np.log(y_out)
            y_out_log[y_out_log == -np.inf] = 0
            popt_background, pcov_background = curve_fit(background, x_out, y_out_log, p0 = [y_out_log[0], 0.2], bounds = ([0, 0], [np.inf, np.inf]))
            background_fit = np.exp(background(x_out_smooth, *popt_background))

            # fitting signal region with substracted background
            popt, pcov = curve_fit(gauss, x, y - np.exp(background(x, *popt_background)), p0 = [calibration_guess, min_vem, std], bounds = ([0, 0, 0], [np.inf, np.inf, np.inf]) )
            model_fit = gauss(x_smooth, *popt) + np.exp(background(x_smooth, *popt_background))
            peak_estimate = popt[0]
                    

            label = r"$\mathrm{VEM}_\mathrm{Peak} = $" if mode == "peak" else r"$\mathrm{VEM}_\mathrm{Charge} = $"
            label += f"{peak_estimate:.3f} +- {np.sqrt(pcov[0][0]):.2e}"

            axes[j].plot(x_smooth, model_fit, label = label, c = colors[j], ls = "--", lw = 2)
            axes[j].plot(x_out_smooth, background_fit, c = "gray", ls = "--", lw = 2)
            axes[j].legend(title = f"Station {name.capitalize()} - PMT #{j + 1}")
            axes[j].set_yscale("log")

            axes[j].set_ylim(10e-1, 1.1 * max(n))

            axes[-1].set_xlabel("VEM peak")

            # if peak_estimate <= min_vem: raise SignalError

            vem_peak.append(peak_estimate)

        peak_evolution.append(vem_peak)

        axes[-1].set_xlabel("VEM peak")

    except SignalError:

        for increment in range(combine_files):

            filename = f"{station_name}/{station_name}_randoms{(index + increment).zfill(4)}.csv"
            # os.system(f"mv /cr/tempdata01/filip/iRODS/{station_name}/{filename} /cr/tempdata01/filip/iRODS/faulty_estimate/{filename}")
            print(f"mv /cr/tempdata01/filip/iRODS/{station_name}/{filename} /cr/tempdata01/filip/iRODS/faulty_estimate/{filename}")

    break



# For VEM Charge

In [ ]:
peak_evolution = []
trigger_examples = []
cut_threshold = 30
leading_bins = 20
trailing_bins = 49
multiplicity = 1
dead_time = trailing_bins
combine_files = 30
station_name = "nuria"

mode = "charge"

if mode == "peak":

    x_range = (cut_threshold, 1200)
    i, f = 20, 100
    func = max
    min_vem = 120
    calibration_guess = GLOBAL.q_peak
    std = 10

elif mode == "charge":

    x_range = (cut_threshold, 12000)
    i, f = 20, 70
    func = sum
    min_vem = 1000
    calibration_guess = GLOBAL.q_charge
    std = 200

first_trigger = lambda x : np.argmax(x > cut_threshold)

def gauss(x, mu, A, sigma):

    return A * np.exp( -(x - mu)**2 / (2 * sigma**2) )

def background(x, A, c):

    return A - (x - min_vem) * c

colors = ["steelblue", "orange", "green"]

h1, h2, h3 = [], [], []

for index in range(0, len(os.listdir(f"/cr/tempdata01/filip/iRODS/{station_name}/")), 3):

    for increment in range(combine_files):

        Buffer = RandomTrace(station_name, index + increment)
        
        vem_peak = []

        # print(f"{index + increment}/{Buffer.all_n_files} {Buffer.random_file}: {len(Buffer._these_traces)} traces", end = "\r")

        station = np.array(Buffer._these_traces)
        pmt1, pmt2, pmt3 = station[:,0], station[:,1], station[:,2]
        cut_mask = np.logical_and(np.any(pmt1 > cut_threshold, axis = 1), np.any(pmt2 > cut_threshold, axis = 1), np.any(pmt3 > cut_threshold, axis = 1))

        for (p1, p2, p3) in zip(pmt1[cut_mask], pmt2[cut_mask], pmt3[cut_mask]):

            steps = iter(range(min([first_trigger(p1), first_trigger(p2), first_trigger(p3)]), 2048))
            # STRONG COINCIDENCE
            for step in steps:

                trigger_list = [p1[step] > cut_threshold, p2[step] > cut_threshold, p3[step] > cut_threshold]

                if trigger_list.count(True) >= multiplicity:

                    start, stop = max(0, step - leading_bins), min(2048, step + trailing_bins)

                    p1_data, p2_data, p3_data = p1[start : stop], p2[start : stop], p3[start : stop]
                    h1.append(func(p1_data)), h2.append(func(p2_data)), h3.append(func(p3_data))

                    if len(trigger_examples) < 100: trigger_examples.append([p1_data, p2_data, p3_data])

                    for _ in range(dead_time): next(steps, None)
                    # step += dead_time       # keep iterating
                    # break                     # stop after first hit
        
            # # WEAK COINCIDENCE
            # trigger_bin = np.argmax(p1 > cut_threshold)
            # start = max(0, trigger_bin - leading_bins)
            # stop = min(trigger_bin + trailing_bins, 2048)

            # p1_data, p2_data, p3_data = p1[start : stop], p2[start : stop], p3[start : stop]

            # if np.any(p2_data > cut_threshold) and np.any(p3_data > cut_threshold):

            #     h1.append(func(p1_data)), h2.append(func(p2_data)), h3.append(func(p3_data))

    fig, axes = plt.subplots(3, 1, sharex = True)
    name = Buffer.random_file.split("_")[0]

    try:
        for j, histogram in enumerate([h1, h2, h3]):

            n, bins, _ = axes[j].hist(histogram,  bins = 250, range = x_range, label = f"{len(histogram)} coincidences", ls = "solid", histtype = "step", color = colors[j], lw = 1)
            bin_centers = 0.5 * (bins[1:] + bins[:-1])

            # Signal region
            x, y = bin_centers[i : f], n[i : f]
            x_smooth = np.linspace(min(x), max(x), 100)

            x_out = list(bin_centers[0 : i]) + list(bin_centers[f : -1])
            y_out = list(n[0 : i]) + list(n[f : -1])
            x_out_smooth = np.linspace(min(x_out), max(x_out), 100)

            # fitting background exponential first
            y_out_log = np.log(y_out)
            y_out_log[y_out_log == -np.inf] = 0
            popt_background, pcov_background = curve_fit(background, x_out, y_out_log, p0 = [y_out_log[0], 0.2], bounds = ([0, 0], [np.inf, np.inf]))
            background_fit = np.exp(background(x_out_smooth, *popt_background))

            # fitting signal region with substracted background
            popt, pcov = curve_fit(gauss, x, y - np.exp(background(x, *popt_background)), p0 = [calibration_guess, min_vem, std], bounds = ([0, 0, 0], [np.inf, np.inf, np.inf]) )
            model_fit = gauss(x_smooth, *popt) + np.exp(background(x_smooth, *popt_background))
            peak_estimate = popt[0]
                    

            label = r"$\mathrm{VEM}_\mathrm{Peak} = $" if mode == "peak" else r"$\mathrm{VEM}_\mathrm{Charge} = $"
            label += f"{peak_estimate:.3f} +- {np.sqrt(pcov[0][0]):.2e}"

            axes[j].plot(x_smooth, model_fit, label = label, c = colors[j], ls = "--", lw = 2)
            axes[j].plot(x_out_smooth, background_fit, c = "gray", ls = "--", lw = 2)
            axes[j].legend(title = f"Station {name.capitalize()} - PMT #{j + 1}")
            axes[j].set_yscale("log")

            axes[j].set_ylim(10e-1, 1.1 * max(n))

            axes[-1].set_xlabel("VEM peak")

            # if peak_estimate <= min_vem: raise SignalError

            vem_peak.append(peak_estimate)

        peak_evolution.append(vem_peak)

        axes[-1].set_xlabel("VEM peak")

    except SignalError:

        for increment in range(combine_files):

            filename = f"{station_name}/{station_name}_randoms{(index + increment).zfill(4)}.csv"
            # os.system(f"mv /cr/tempdata01/filip/iRODS/{station_name}/{filename} /cr/tempdata01/filip/iRODS/faulty_estimate/{filename}")
            print(f"mv /cr/tempdata01/filip/iRODS/{station_name}/{filename} /cr/tempdata01/filip/iRODS/faulty_estimate/{filename}")

    break

In [ ]:
for (pmt1, pmt2, pmt3) in trigger_examples:

    plt.plot(range(len(pmt1)), pmt1)
    plt.plot(range(len(pmt2)), pmt2)
    plt.plot(range(len(pmt3)), pmt3)
    plt.axvline(20, ls = "--", c = "k")
    plt.axhline(30, ls = ":", c = "steelblue", alpha = 0.2)

    plt.xlim(0, 69)

    plt.show()

    input()
    plt.clf()